In [ ]:
from tqdm import notebook
import dimod
from dimod import BinaryQuadraticModel
import dwave
from dwave.samplers import SimulatedAnnealingSampler
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
import numpy as np

In [ ]:
def annealing(H,num_shot,quantum=False,sampler=None,embedding=None,\
              anneal_time=None,chain_strength=10):
    if quantum == False:
        sampler_  = SimulatedAnnealingSampler()
        sampleset = sampler_.sample(H,num_reads=num_shot,answer_mode='raw')
    elif quantum == True and type(sampler) != type(None) and type(embedding) != type(None):
        sampler_     = FixedEmbeddingComposite(sampler,embedding)
        sampleset    = sampler_.sample(H,num_reads=num_shot,chain_strength=chain_strength,\
                                    answer_mode='raw',annealing_time=anneal_time)
    return sampleset

def hamiltonian(A,h):
    linear    = {}
    quadratic = {}
    N = A.shape[0]
    for n in range(N):
        linear[n] = h[n]
    for n in range(N):
        for m in range(N):
            quadratic[(n,m)] = A[n,m]
    H = dimod.BinaryQuadraticModel(linear,quadratic,0,vartype='SPIN')
    return H

def retEnergy(A,X):
    return np.dot(X,A@X)/np.dot(X,X)

class Calc:
    def __init__(self,A,quantum,iteration,embd,sampler,num_shot=100,anneal_time=800):
        self.A             = A
        self.quantum       = quantum
        self.N             = A.shape[0]
        self.iteration     = iteration
        self.embd          = embd
        self.sampler       = sampler
        self.beta          = 0.9
        self.num_shot      = num_shot
        self.anneal_time   = anneal_time
        self.scale         = 2.
        self.sampleset_lst = []
        self.c_lst         = []
        self.d_lst         = []  
        self.E_lst         = []
        self.X_lst         = []
    def calc(self):
        v         = 0
        h         = np.zeros((self.N,))
        H         = hamiltonian(self.A,h)
        ch        = self.scale * np.max(np.abs(self.A))
        sampleset = annealing(H,self.num_shot,quantum=self.quantum,\
                              sampler=self.sampler,embedding=self.embd,\
                                  anneal_time=self.anneal_time,chain_strength=ch)
        self.sampleset_lst.append(sampleset)
        confi     = sampleset.first.sample
        X         = np.array([confi[n]+0. for n in range(self.N)])
        E         = retEnergy(self.A,X)
        E0        = E
        E__       = E
        X_        = X    
        self.E_lst.append(E)
        self.X_lst.append(X_)
#         for idx in notebook.tqdm(range(self.iteration)):  
        for idx in range(self.iteration):
            h = 2 * self.A @ X_ - 2 * E * X_
            c = np.random.uniform(0.9,1.1,size=self.N) 
            d = np.random.uniform(-1.1,-0.9,size=self.N) 
            self.c_lst.append(c)
            self.d_lst.append(d)
#             c = np.ones((self.N,))
#             d = -np.ones((self.N,))
            a = (c - d) / 2
            b = (c + d) / 2
            A_ = np.zeros((self.N,self.N))
            for n in range(self.N):
                for m in range(self.N):
                    A_[n,m] = a[n] * A[n,m] * a[m]
            h_        = (2 * A @ b + h) * a
            H         = hamiltonian(A_,h_)  
            mxA = np.max(np.abs(A))
            mxh = np.max(np.abs(h))
            mx  = max([mxA,mxh])
            ch = self.scale * mx
            sampleset = annealing(H,self.num_shot,quantum=self.quantum,\
                                  sampler=self.sampler,embedding=self.embd,\
                                      anneal_time=self.anneal_time,chain_strength=ch)
            self.sampleset_lst.append(sampleset)
            confi     = sampleset.first.sample    
            X         = np.array([confi[n]+0. for n in range(self.N)])
            X         = np.where(X==1,c,d)
            v         = self.beta * v + X
            X_        = X_ + v
            E         = retEnergy(self.A,X_)
            self.E_lst.append(E)
            self.X_lst.append(X_)
            if E0 < E and idx > 50:
                break
        data              = {}
        data['E']         = self.E_lst
        data['X']         = self.X_lst
        data['c']         = self.c_lst
        data['d']         = self.d_lst
        data['sampleset'] = self.sampleset_lst
        return data
    
class Calc_rev:
    def __init__(self,A,quantum,iteration,embd,sampler,num_shot=100,anneal_time=800):
        self.A             = A
        self.quantum       = quantum
        self.N             = A.shape[0]
        self.iteration     = iteration
        self.embd          = embd
        self.sampler       = sampler
        self.beta          = 0.9
        self.num_shot      = num_shot
        self.anneal_time   = anneal_time
        self.scale         = 2.
        self.sampleset_lst = []
        self.c_lst         = []
        self.d_lst         = []  
        self.E_lst         = []
        self.X_lst         = []
    def calc(self):
        v         = 0
        h         = np.zeros((self.N,))
        H         = hamiltonian(-self.A,h)
        ch        = self.scale * np.max(np.abs(self.A))
        sampleset = annealing(H,self.num_shot,quantum=self.quantum,\
                              sampler=self.sampler,embedding=self.embd,\
                                  anneal_time=self.anneal_time,chain_strength=ch)
        self.sampleset_lst.append(sampleset)
        confi     = sampleset.first.sample
        X         = np.array([confi[n]+0. for n in range(self.N)])
        E         = retEnergy(self.A,X)
        E0        = E        
        X_        = X    
        self.E_lst.append(E)
        self.X_lst.append(X_)
#         for idx in notebook.tqdm(range(self.iteration)):  
        for idx in range(self.iteration):
            h = 2 * self.A @ X_ - 2 * E * X_
            c = np.random.uniform(0.9,1.1,size=self.N) 
            d = np.random.uniform(-1.1,-0.9,size=self.N) 
            self.c_lst.append(c)
            self.d_lst.append(d)
#             c = np.ones((self.N,))
#             d = -np.ones((self.N,))
            a = (c - d) / 2
            b = (c + d) / 2
            A_ = np.zeros((self.N,self.N))
            for n in range(self.N):
                for m in range(self.N):
                    A_[n,m] = a[n] * A[n,m] * a[m]
            h_        = (2 * A @ b + h) * a
            H         = hamiltonian(-A_,-h_)  
            mxA = np.max(np.abs(A))
            mxh = np.max(np.abs(h))
            mx  = max([mxA,mxh])
            ch = self.scale * mx
            sampleset = annealing(H,self.num_shot,quantum=self.quantum,\
                                  sampler=self.sampler,embedding=self.embd,\
                                      anneal_time=self.anneal_time,chain_strength=ch)
            self.sampleset_lst.append(sampleset)
            confi     = sampleset.first.sample    
            X         = np.array([confi[n]+0. for n in range(self.N)])
            X         = np.where(X==1,c,d)
            v         = self.beta * v + X
            X_        = X_ + v
            E         = retEnergy(self.A,X_)
            self.E_lst.append(E)
            self.X_lst.append(X_)
            if E0 > E and idx > 20:
                break
        data              = {}
        data['E']         = self.E_lst
        data['X']         = self.X_lst
        data['c']         = self.c_lst
        data['d']         = self.d_lst
        data['sampleset'] = self.sampleset_lst
        return data
    
def retR(A):
    N = A.shape[0]
    R = np.zeros((N,))
    for i in np.arange(N):
        r    = np.sum([np.abs(A[i][j]) for j in np.arange(N) if j != i])
        R[i] = r
    return R        
###############################################################################
###############################################################################
def calc_full_spec(A,quantum,iteration,embd,sampler,num_shot,anneal_time):
    Data_lst = []
    N        = A.shape[0]
    E_lst    = np.zeros((N,))
    X_lst    = np.zeros((N,N))
    cc       = Calc(A,quantum,iteration,embd,sampler,num_shot,anneal_time)
    data     = cc.calc()
    idx      = np.argmin(data['E'])
    E0, X0   = data['E'][idx], data['X'][idx]
    Data_lst.append(data)
    X0      /= np.linalg.norm(X0)  
    cc       = Calc_rev(A,quantum,iteration,embd,sampler,num_shot,anneal_time)
    data     = cc.calc()
    idx      = np.argmax(data['E'])
    EN, XN   = data['E'][idx], data['X'][idx]
    Data_lst.append(data)
    XN      /= np.linalg.norm(XN) 
    
    gap_   = EN - E0    
    R      = retR(A)
    w      = (np.max(R) - np.min(R))
    w      = gap_ * np.where(w<1,1,w)    
    
    E_lst[0]  = E0
    X_lst[0]  = X0
    E_lst[-1] = EN
    X_lst[-1] = XN    
    X         = X0    
    for n in notebook.tqdm(range(1,N-1),desc='iter'):
        A_       = A + w * X.reshape(-1,1) @ X.reshape(1,-1)
        cc       = Calc(A_,quantum,iteration,embd,sampler,num_shot,anneal_time)
        data     = cc.calc()
        idx      = np.argmin(data['E'])
        E, X     = data['E'][idx], data['X'][idx]
        X       /= np.linalg.norm(X)
        E_lst[n] = E
        X_lst[n] = X
        Data_lst.append(data)
        A           = A_
    return Data_lst, E_lst, X_lst  

In [ ]:
sampler_zephyr = DWaveSampler(solver='Advantage2_prototype2.3',\
                                token='---')
print("Connected to sampler", sampler_zephyr.solver.name)
zephyr_graph = sampler_zephyr.to_networkx_graph()

In [ ]:
quantum     = True
num_shot    = 100
iteration   = 100
anneal_time = 100
A           = 
embd        = embd
Data, E_lst, X_lst = calc_full_spec(A,quantum,iteration,embd,sampler,num_shot,anneal_time)